In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
import os
from decimal import Decimal
import glob
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
import platform
from matplotlib import font_manager, rc
font = 'Malgun Gothic'
plt.rcParams['font.family'] = font
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data= pd.read_csv('total_data.csv')

In [3]:
start_year=2009
end_year=2018
TT_year=2016

In [4]:
# data_C = pd.read_excel('Zone_FS_2016_C.xlsx')
data=data[(data['Year']>=start_year) & (data['Year']<=end_year)]
data.index=range(len(data))

In [5]:
data_train=data[data['Year']<=TT_year]
data_train.index=range(len(data_train))
data_test=data[data['Year']>TT_year]
data_test.index=range(len(data_test))

# SVM

In [6]:
result_SVM=pd.read_excel('./final_total_result/result_SVM_outbreak.xlsx')
predict_SVM=pd.read_excel('./final_total_result/predict_SVM_outbreak.xlsx')

In [7]:
result_sum=result_SVM.iloc[:,result_SVM.columns.str.contains("Sim")]
predict_sum=predict_SVM.iloc[:,predict_SVM.columns.str.contains("Sim")]

In [8]:
result_sum['warning']=result_SVM['warning']
predict_sum['warning']=predict_SVM['warning']


In [9]:
r_tmp=result_SVM.iloc[:,result_SVM.columns.str.contains("Sim")]
result_sum['SVM_mean']=r_tmp.mean(axis=1)
result_sum['SVM_95+']=r_tmp.quantile(axis=1,q=0.975)
result_sum['SVM_95-']=r_tmp.quantile(axis=1,q=0.025)

In [10]:
p_tmp=predict_SVM.iloc[:,predict_SVM.columns.str.contains("Sim")]
predict_sum['SVM_mean']=p_tmp.mean(axis=1)
predict_sum['SVM_95+']=p_tmp.quantile(axis=1,q=0.975)
predict_sum['SVM_95-']=p_tmp.quantile(axis=1,q=0.025)

In [11]:
p_v=0.5

for i in result_sum.index:
    if (result_sum.loc[i,'SVM_95+']>=p_v):
        result_sum.loc[i,'prediction_95+']=1
    else:
        result_sum.loc[i,'prediction_95+']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'SVM_95+']>=p_v):
        predict_sum.loc[i,'prediction_95+']=1
    else:
        predict_sum.loc[i,'prediction_95+']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'SVM_95-']>=p_v):
        result_sum.loc[i,'prediction_95-']=1
    else:
        result_sum.loc[i,'prediction_95-']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'SVM_95-']>=p_v):
        predict_sum.loc[i,'prediction_95-']=1
    else:
        predict_sum.loc[i,'prediction_95-']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'SVM_mean']>=p_v):
        result_sum.loc[i,'prediction']=1
    else:
        result_sum.loc[i,'prediction']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'SVM_mean']>=p_v):
        predict_sum.loc[i,'prediction']=1
    else:
        predict_sum.loc[i,'prediction']=0

In [12]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [13]:
result_save['Week']=data_train['Week']
result_save['Year']=data_train['Year']
result_save['season']=data_train['season']

predict_save['Week']=data_test['Week']
predict_save['Year']=data_test['Year']
predict_save['season']=data_test['season']

In [14]:
for i in range(len(result_save)):
    temp_sum=result_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        result_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            result_save.loc[i,'level']=1
        else:
            result_save.loc[i,'level']=2

In [15]:
for i in range(len(predict_save)):
    temp_sum=predict_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        predict_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            predict_save.loc[i,'level']=1
        else:
            predict_save.loc[i,'level']=2

In [16]:
result_save.to_excel('level_SVM_train.xlsx')
predict_save.to_excel('level_SVM_test.xlsx')

# MLP

In [17]:
result_MLP=pd.read_excel('./final_total_result/result_MLP_outbreak.xlsx')
predict_MLP=pd.read_excel('./final_total_result/predict_MLP_outbreak.xlsx')

In [18]:
result_sum=result_MLP.iloc[:,result_MLP.columns.str.contains("Sim")]
predict_sum=predict_MLP.iloc[:,predict_MLP.columns.str.contains("Sim")]

In [19]:
result_sum['warning']=result_MLP['warning']
predict_sum['warning']=predict_MLP['warning']


In [20]:
r_tmp=result_MLP.iloc[:,result_MLP.columns.str.contains("Sim")]
result_sum['MLP_mean']=r_tmp.mean(axis=1)
result_sum['MLP_95+']=r_tmp.quantile(axis=1,q=0.975)
result_sum['MLP_95-']=r_tmp.quantile(axis=1,q=0.025)

In [21]:
p_tmp=predict_MLP.iloc[:,predict_MLP.columns.str.contains("Sim")]
predict_sum['MLP_mean']=p_tmp.mean(axis=1)
predict_sum['MLP_95+']=p_tmp.quantile(axis=1,q=0.975)
predict_sum['MLP_95-']=p_tmp.quantile(axis=1,q=0.025)

In [22]:
p_v=0.5

for i in result_sum.index:
    if (result_sum.loc[i,'MLP_95+']>=p_v):
        result_sum.loc[i,'prediction_95+']=1
    else:
        result_sum.loc[i,'prediction_95+']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'MLP_95+']>=p_v):
        predict_sum.loc[i,'prediction_95+']=1
    else:
        predict_sum.loc[i,'prediction_95+']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'MLP_95-']>=p_v):
        result_sum.loc[i,'prediction_95-']=1
    else:
        result_sum.loc[i,'prediction_95-']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'MLP_95-']>=p_v):
        predict_sum.loc[i,'prediction_95-']=1
    else:
        predict_sum.loc[i,'prediction_95-']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'MLP_mean']>=p_v):
        result_sum.loc[i,'prediction']=1
    else:
        result_sum.loc[i,'prediction']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'MLP_mean']>=p_v):
        predict_sum.loc[i,'prediction']=1
    else:
        predict_sum.loc[i,'prediction']=0

In [23]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [24]:
result_save['Week']=data_train['Week']
result_save['Year']=data_train['Year']
result_save['season']=data_train['season']

predict_save['Week']=data_test['Week']
predict_save['Year']=data_test['Year']
predict_save['season']=data_test['season']

In [25]:
for i in range(len(result_save)):
    temp_sum=result_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        result_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            result_save.loc[i,'level']=1
        else:
            result_save.loc[i,'level']=2

In [26]:
for i in range(len(predict_save)):
    temp_sum=predict_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        predict_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            predict_save.loc[i,'level']=1
        else:
            predict_save.loc[i,'level']=2

In [27]:
result_save.to_excel('level_MLP_train.xlsx')
predict_save.to_excel('level_MLP_test.xlsx')

# RF

In [28]:
result_RF=pd.read_excel('./final_total_result/result_RF_outbreak.xlsx')
predict_RF=pd.read_excel('./final_total_result/predict_RF_outbreak.xlsx')

In [29]:
result_sum=result_RF.iloc[:,result_RF.columns.str.contains("Sim")]
predict_sum=predict_RF.iloc[:,predict_RF.columns.str.contains("Sim")]

In [30]:
result_sum['warning']=result_MLP['warning']
predict_sum['warning']=predict_MLP['warning']


In [31]:
r_tmp=result_MLP.iloc[:,result_MLP.columns.str.contains("Sim")]
result_sum['MLP_mean']=r_tmp.mean(axis=1)
result_sum['MLP_95+']=r_tmp.quantile(axis=1,q=0.975)
result_sum['MLP_95-']=r_tmp.quantile(axis=1,q=0.025)

In [32]:
p_tmp=predict_MLP.iloc[:,predict_MLP.columns.str.contains("Sim")]
predict_sum['MLP_mean']=p_tmp.mean(axis=1)
predict_sum['MLP_95+']=p_tmp.quantile(axis=1,q=0.975)
predict_sum['MLP_95-']=p_tmp.quantile(axis=1,q=0.025)

In [33]:
p_v=0.5

for i in result_sum.index:
    if (result_sum.loc[i,'MLP_95+']>=p_v):
        result_sum.loc[i,'prediction_95+']=1
    else:
        result_sum.loc[i,'prediction_95+']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'MLP_95+']>=p_v):
        predict_sum.loc[i,'prediction_95+']=1
    else:
        predict_sum.loc[i,'prediction_95+']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'MLP_95-']>=p_v):
        result_sum.loc[i,'prediction_95-']=1
    else:
        result_sum.loc[i,'prediction_95-']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'MLP_95-']>=p_v):
        predict_sum.loc[i,'prediction_95-']=1
    else:
        predict_sum.loc[i,'prediction_95-']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'MLP_mean']>=p_v):
        result_sum.loc[i,'prediction']=1
    else:
        result_sum.loc[i,'prediction']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'MLP_mean']>=p_v):
        predict_sum.loc[i,'prediction']=1
    else:
        predict_sum.loc[i,'prediction']=0

In [34]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [35]:
result_save['Week']=data_train['Week']
result_save['Year']=data_train['Year']
result_save['season']=data_train['season']

predict_save['Week']=data_test['Week']
predict_save['Year']=data_test['Year']
predict_save['season']=data_test['season']

In [36]:
for i in range(len(result_save)):
    temp_sum=result_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        result_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            result_save.loc[i,'level']=1
        else:
            result_save.loc[i,'level']=2

In [37]:
for i in range(len(predict_save)):
    temp_sum=predict_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        predict_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            predict_save.loc[i,'level']=1
        else:
            predict_save.loc[i,'level']=2

In [38]:
result_save.to_excel('level_RF_train.xlsx')
predict_save.to_excel('level_RF_test.xlsx')

# GB

In [39]:
result_GB=pd.read_excel('./final_total_result/result_GB_outbreak.xlsx')
predict_GB=pd.read_excel('./final_total_result/predict_GB_outbreak.xlsx')

In [40]:
result_sum=result_GB.iloc[:,result_GB.columns.str.contains("Sim")]
predict_sum=predict_GB.iloc[:,predict_GB.columns.str.contains("Sim")]

In [41]:
result_sum['warning']=result_GB['warning']
predict_sum['warning']=predict_GB['warning']


In [42]:
r_tmp=result_GB.iloc[:,result_GB.columns.str.contains("Sim")]
result_sum['GB_mean']=r_tmp.mean(axis=1)
result_sum['GB_95+']=r_tmp.quantile(axis=1,q=0.975)
result_sum['GB_95-']=r_tmp.quantile(axis=1,q=0.025)

In [43]:
p_tmp=predict_GB.iloc[:,predict_GB.columns.str.contains("Sim")]
predict_sum['GB_mean']=p_tmp.mean(axis=1)
predict_sum['GB_95+']=p_tmp.quantile(axis=1,q=0.975)
predict_sum['GB_95-']=p_tmp.quantile(axis=1,q=0.025)

In [44]:
p_v=0.5

for i in result_sum.index:
    if (result_sum.loc[i,'GB_95+']>=p_v):
        result_sum.loc[i,'prediction_95+']=1
    else:
        result_sum.loc[i,'prediction_95+']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'GB_95+']>=p_v):
        predict_sum.loc[i,'prediction_95+']=1
    else:
        predict_sum.loc[i,'prediction_95+']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'GB_95-']>=p_v):
        result_sum.loc[i,'prediction_95-']=1
    else:
        result_sum.loc[i,'prediction_95-']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'GB_95-']>=p_v):
        predict_sum.loc[i,'prediction_95-']=1
    else:
        predict_sum.loc[i,'prediction_95-']=0
        
for i in result_sum.index:
    if (result_sum.loc[i,'GB_mean']>=p_v):
        result_sum.loc[i,'prediction']=1
    else:
        result_sum.loc[i,'prediction']=0

for i in predict_sum.index:
    if (predict_sum.loc[i,'GB_mean']>=p_v):
        predict_sum.loc[i,'prediction']=1
    else:
        predict_sum.loc[i,'prediction']=0

In [45]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [46]:
result_save['Week']=data_train['Week']
result_save['Year']=data_train['Year']
result_save['season']=data_train['season']

predict_save['Week']=data_test['Week']
predict_save['Year']=data_test['Year']
predict_save['season']=data_test['season']

In [47]:
for i in range(len(result_save)):
    temp_sum=result_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        result_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            result_save.loc[i,'level']=1
        else:
            result_save.loc[i,'level']=2

In [48]:
for i in range(len(predict_save)):
    temp_sum=predict_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        predict_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            predict_save.loc[i,'level']=1
        else:
            predict_save.loc[i,'level']=2

In [49]:
result_save.to_excel('level_GB_train.xlsx')
predict_save.to_excel('level_GB_test.xlsx')

# LSTM

In [51]:
result_LSTM=pd.read_excel('./final_total_result/result_save.xlsx')
predict_LSTM=pd.read_excel('./final_total_result/predict_save.xlsx')

In [53]:
result_sum=result_LSTM
predict_sum=predict_LSTM


In [54]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [55]:
result_save['Week']=data_train['Week']
result_save['Year']=data_train['Year']
result_save['season']=data_train['season']

predict_save['Week']=data_test['Week']
predict_save['Year']=data_test['Year']
predict_save['season']=data_test['season']

In [56]:
for i in range(len(result_save)):
    temp_sum=result_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        result_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            result_save.loc[i,'level']=1
        else:
            result_save.loc[i,'level']=2

In [57]:
for i in range(len(predict_save)):
    temp_sum=predict_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        predict_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            predict_save.loc[i,'level']=1
        else:
            predict_save.loc[i,'level']=2

In [58]:
result_save.to_excel('level_LSTM_train.xlsx')
predict_save.to_excel('level_LSTM_test.xlsx')

# GRU

# GRU

In [62]:
result_GRU=pd.read_excel('./final_total_result/result_save_gru.xlsx')
predict_GRU=pd.read_excel('./final_total_result/predict_save_gru.xlsx')

In [63]:
result_sum=result_GRU
predict_sum=predict_GRU


In [64]:
result_save=result_sum[['warning', 'prediction_95-','prediction','prediction_95+']]
predict_save=predict_sum[['warning', 'prediction_95-','prediction','prediction_95+']]

In [65]:
result_save['Week']=data_train['Week']
result_save['Year']=data_train['Year']
result_save['season']=data_train['season']

predict_save['Week']=data_test['Week']
predict_save['Year']=data_test['Year']
predict_save['season']=data_test['season']

In [66]:
for i in range(len(result_save)):
    temp_sum=result_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        result_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            result_save.loc[i,'level']=1
        else:
            result_save.loc[i,'level']=2

In [67]:
for i in range(len(predict_save)):
    temp_sum=predict_save.loc[i,['prediction_95-','prediction','prediction_95+']].sum()
    if temp_sum==3:
        predict_save.loc[i,'level']=3
    else:
        if temp_sum==0:
            predict_save.loc[i,'level']=1
        else:
            predict_save.loc[i,'level']=2

In [68]:
result_save.to_excel('level_GRU_train.xlsx')
predict_save.to_excel('level_GRU_test.xlsx')